## Chapter 7: The Linear Regression Model

In [1]:
import pandas as pd
import numpy as np

from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf

import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

In [16]:
df = pd.read_csv("bookprogs/olsexamp.dat", sep=' ', header=None, index_col=[0])
df.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13
0,,,,,,,,,,,,,
1,1,53,1,1,14,110.00,0,0,0,28,31,13,4
2,1,33,1,0,16,26.25,0,0,0,18,11,9,3
3,1,24,1,0,14,32.50,0,0,0,19,16,8,2
4,1,19,1,0,12,110.00,0,0,0,17,13,11,2
5,1,40,1,0,18,82.50,0,0,0,21,19,12,2


In [3]:
# MH sampling of parameters in linear regression
#number of iterations
m=20000

#read in data, establish x and y matrices 
x = df.loc[1:2313, np.arange(1,10)].values
y = df.loc[1:2313, 10].values

In [4]:
%%time
# establish parameter vectors, proposal scales and acceptance rates
s2 = np.ones(m);
b = np.zeros((m,9))
results = sm.OLS(y,x).fit()
bscale = np.sqrt(np.diag(results.cov_params()))*0.5
s2scale = np.std(results.resid*(2313-1)/(2313-9), ddof=1)*0.5
accrate = np.zeros((m,9)) 
s2accrate = np.zeros(m)

#unnormalized posterior distribution function
def post(x,y,b,s2):
    eps = y - np.dot(x,b)
    return -1157.5*np.log(s2) + (-0.5/s2) * np.dot(eps, eps)


# write output to file and screen
with open('ols_examp.csv', 'w+') as file:

    # Begin MH Sampling
    for i in range(1,m):
        # temporarily set new values of b
        b[i] = b[i-1] 

        # update regression parameters
        for j in range(9):
            # generate candidate and assume it will be accepted...
            b[i,j] = b[i-1,j] + np.random.normal(loc=0, scale=bscale[j])
            acc=1

            #...until it is evaluated for rejection
            if(post(x,y,b[i],s2[i-1]) - post(x,y,b[i-1],s2[i-1]) < np.log(np.random.uniform())):
                b[i,j] = b[i-1,j]
                acc = 0
            accrate[i,j] = (accrate[i-1,j]*(i-1)+acc)/i


        # update s2.  generate candidate and assume accepted
        s2[i] = s2[i-1] + np.random.normal(loc=0, scale=s2scale); 
        acc = 1

        #...until it is evaluated for rejection
        if (s2[i]<0 or (post(x,y,b[i],s2[i]) - post(x,y,b[i],s2[i-1])) < np.log(np.random.uniform())):
            s2[i] = s2[i-1] 
            acc = 0
        s2accrate[i] = (s2accrate[i-1]*(i-1)+acc)/i
        
        # Write to file
        file.write(' '.join(['%f'%F for F in b[i]]))
        file.write(str(s2[i])+' ')
        file.write(' '.join(['%f'%F for F in accrate[i]]))
        file.write(str(s2accrate[i])+'\n')

        if (i%100==0):
            print(i,'\t{0:4f}\t{1:4f}\t{2:4f}\t{3:4f}'.format(b[i,0],s2[i],accrate[i,0],s2accrate[i]))

100 	11.157221	26.500208	0.520000	0.530000
200 	16.011826	22.431028	0.485000	0.455000
300 	18.075734	20.733817	0.440000	0.376667
400 	18.703320	21.531166	0.420000	0.377500
500 	19.683405	20.693474	0.410000	0.378000
600 	18.923182	22.049237	0.410000	0.363333
700 	19.575884	21.379060	0.400000	0.357143
800 	19.348980	22.134716	0.400000	0.346250
900 	20.177120	21.849069	0.398889	0.343333
1000 	20.717929	21.903968	0.391000	0.343000
1100 	21.059797	21.123481	0.387273	0.340000
1200 	20.277742	21.607045	0.388333	0.342500
1300 	19.161954	20.921461	0.387692	0.337692
1400 	21.010911	22.671986	0.385000	0.341429
1500 	19.806686	20.158059	0.390000	0.340000
1600 	19.595146	19.925806	0.387500	0.338750
1700 	19.569459	21.739316	0.388824	0.334118
1800 	19.485581	20.780026	0.390556	0.334444
1900 	19.371416	22.547336	0.386842	0.337895
2000 	19.413337	21.727079	0.385500	0.335500
2100 	20.033061	21.865798	0.387619	0.331429
2200 	20.342013	21.615311	0.395000	0.329545
2300 	19.986558	20.699160	0.393043	0.3304

18700 	19.830068	21.142798	0.381818	0.316631
18800 	20.117382	22.012400	0.381809	0.316489
18900 	20.087193	21.620090	0.382381	0.316508
19000 	19.990611	20.522511	0.382579	0.316263
19100 	19.681576	21.568598	0.382618	0.316178
19200 	19.875655	20.388520	0.382448	0.316615
19300 	20.548198	22.863138	0.382280	0.316528
19400 	20.646801	20.544089	0.382474	0.316804
19500 	20.298728	21.302982	0.382359	0.316205
19600 	19.783106	21.951283	0.382143	0.316378
19700 	19.240223	21.768913	0.382081	0.316701
19800 	18.923125	21.614372	0.381768	0.316566
19900 	18.528905	21.059512	0.381859	0.316533
Wall time: 47.9 s


---

In [5]:
%%time
# Gibbs sampling from full conditionals in OLS example

#number of iterations
m=20000

# read only observations with complete information, n=2313
x = df.loc[1:2313, np.arange(1,10)].values
y = df.loc[1:2313, 10].values

#establish parameter vectors and constant quantities
s2 = np.ones(m) 
b = np.zeros((m,9))
xtxi = np.linalg.inv(np.dot(x.T,x))
results = sm.OLS(y,x).fit()
pars = results.params

# write output to file and screen
with open('ols_examp.csv', 'w+') as file:

    #Gibbs sampling begins
    for i in range(1,m):
        #simulate beta from its multivariate normal conditional
        b[i] = pars + np.dot(np.random.normal(size=9).T, np.linalg.cholesky(s2[i-1]*xtxi).T)

        # simulate sigma from its inverse gamma distribution
        eps = y-np.dot(x,b[i]) 
        s2[i] = stats.invgamma.rvs(a=2313/2, loc=0, scale=0.5*np.dot(eps.T, eps))

        file.write(' '.join(['%f'%F for F in b[i]]))
        file.write(str(s2[i])+'\n')

        if (i%100==0):
            print(i,'\t{0:4f}\t{1:4f}'.format(b[i,0], s2[i]))

100 	20.753008	21.682135
200 	19.471531	21.949193
300 	20.477530	21.197756
400 	20.196056	20.601288
500 	20.774887	21.458489
600 	18.889315	20.927864
700 	19.566814	20.927716
800 	19.494963	22.024607
900 	20.149453	21.875542
1000 	20.336306	21.820339
1100 	19.765013	20.691970
1200 	19.340992	21.528732
1300 	19.760420	21.319494
1400 	19.339396	20.583700
1500 	19.578271	21.265970
1600 	19.510060	21.677599
1700 	19.214547	20.223860
1800 	18.688767	22.043114
1900 	19.625505	20.805722
2000 	18.878604	21.871864
2100 	19.623938	21.584391
2200 	19.023665	20.880050
2300 	20.172910	21.253163
2400 	19.984134	22.080895
2500 	18.911582	21.709142
2600 	19.278928	21.661152
2700 	20.103718	22.580653
2800 	19.433480	20.963657
2900 	19.938896	20.771179
3000 	19.312804	21.128341
3100 	19.238065	22.322148
3200 	19.218612	20.553001
3300 	19.539806	21.247668
3400 	19.052885	21.856742
3500 	19.067106	20.037360
3600 	19.768427	20.745823
3700 	19.983441	22.403239
3800 	19.719978	21.781480
3900 	19.700174	21.60

---

In [6]:
%%time
# Gibbs sampling using composition method in OLS

# number of iterations
m = 20000

# read only observations with complete information, n=2313
x = df.loc[1:2313, np.arange(1,10)].values
y = df.loc[1:2313, 10].values

# establish parameter vectors and constant quantities
s2 = np.ones(m) 
b = np.zeros((m,9))
xtxi = np.linalg.inv(np.dot(x.T,x))
results = sm.OLS(y,x).fit()
pars = results.params

#simulate sigma from its inverse gamma marginal
res = results.resid
s2 = stats.invgamma.rvs(a=(2313-9)/2, loc=0, scale=0.5*np.dot(res.T, res), size=m)

#write output to file and screen
with open('ols_examp.csv', 'w+') as file:
        
    #simulate beta vector from appropriate mvn
    for i in range(m):
        b[i] = pars + np.dot(np.random.normal(size=9).T, np.linalg.cholesky(s2[i]*xtxi).T)

        file.write(' '.join(['%f'%F for F in b[i]]))
        file.write(str(s2[i])+'\n')

        if (i%100==0):
            print(i,'\t{0:4f}\t{1:4f}'.format(b[i,0], s2[i]))

0 	19.207562	20.568897
100 	19.466062	21.674057
200 	20.473919	21.198403
300 	20.160449	21.550946
400 	20.296289	21.855691
500 	19.955432	21.668380
600 	20.769738	22.005645
700 	19.557291	19.997448
800 	20.462699	22.019542
900 	19.924584	21.543368
1000 	20.069447	20.091586
1100 	20.128145	20.678302
1200 	19.981482	21.635281
1300 	18.940698	20.896246
1400 	19.661991	20.468253
1500 	19.904113	21.893817
1600 	19.205007	21.338099
1700 	20.205585	21.094921
1800 	19.978102	21.909611
1900 	19.724255	20.480191
2000 	19.327497	20.071379
2100 	18.417740	21.383151
2200 	20.328294	21.086989
2300 	18.839303	21.776275
2400 	19.634856	21.318128
2500 	19.908206	20.714635
2600 	19.283764	21.127701
2700 	19.704276	21.987285
2800 	20.081646	21.915842
2900 	19.149174	20.282495
3000 	20.216659	20.207725
3100 	19.802902	20.885048
3200 	18.569550	22.143017
3300 	20.486462	20.862864
3400 	19.558555	22.215906
3500 	19.273424	21.004651
3600 	19.584245	22.305965
3700 	20.262596	20.678311
3800 	20.456571	21.19070

---
#### Missing Data

In [25]:
%%time
# MH sampling of parameters in linear regression
# number of iterations
m=20000

#read in data, establish x and y matrices 
x = df[np.arange(1,10)].values
y = df[10].values

# establish parameter vectors
s2 = np.ones(m) 
b = np.zeros((m,9))

#write output to file and screen
with open('ols_examp_miss.csv', 'w+') as file:
        
    #simulate beta vector from appropriate mvn
    for i in range(m):
        xstar = x
        xstar[x[:,5]==999,5] = np.random.normal(loc=np.mean(x[x[:,5]!=999,5]), 
                                                scale=np.std(x[x[:,5]!=999,5]), 
                                                size=np.sum(x[:,5]==999))
        ystar = y
        ystar[y==999] = np.random.normal(loc=np.dot(x[y==999],b[i-1]), 
                                         scale=np.sqrt(s2[i-1]), 
                                         size=np.sum(y==999))
        
        results = sm.OLS(ystar,xstar).fit()
        pars = results.params
        xtxi = np.linalg.inv(np.dot(xstar.T,xstar))
        b[i] = pars + np.dot(np.random.normal(size=9).T, np.linalg.cholesky(s2[i-1]*xtxi).T)
        
        #simulate sigma from its inverse gamma marginal
        res = ystar - np.dot(xstar,b[i])
        s2[i] = stats.invgamma.rvs(a=len(y)/2, scale=0.5*np.dot(res.T, res))

        file.write(' '.join(['%f'%F for F in b[i]]))
        file.write(str(s2[i])+'\n')

        if (i%100==0):
            print(i,'\t{0:4f}\t{1:4f}'.format(b[i,0], s2[i]))

0 	18.397340	30.416370
100 	17.147844	29.539021
200 	17.288901	31.367725
300 	18.233321	29.918033
400 	19.250882	31.045722
500 	18.577674	30.509915
600 	18.511845	31.077468
700 	18.516957	29.913930
800 	18.602991	30.224640
900 	18.546334	29.808450
1000 	17.587637	29.422606
1100 	18.077520	30.091780
1200 	17.967676	28.474717
1300 	16.705492	29.769009
1400 	19.265541	30.404080
1500 	18.648874	30.822299
1600 	18.005415	30.027289
1700 	17.403934	30.647829
1800 	18.432335	30.967013
1900 	18.459462	30.455405
2000 	18.466927	30.204425
2100 	18.367702	29.711901
2200 	17.811734	30.775915
2300 	18.131375	31.156013
2400 	18.781994	29.098751
2500 	17.863249	28.691132
2600 	18.934803	30.383094
2700 	18.739151	30.247540
2800 	18.173710	29.220274
2900 	18.310158	29.054298
3000 	18.775173	30.103919
3100 	18.359890	30.133194
3200 	18.723241	30.133513
3300 	18.740284	29.543400
3400 	18.576087	29.362122
3500 	17.827092	30.366969
3600 	17.131722	31.709558
3700 	18.281197	29.415733
3800 	18.258353	30.69268